## Individual Proposal

#### Broad Question
Question 3: We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability. 

#### Specific Question
Does the background of a player (Age, Experience) predict when and how long they are on the server? 

### Summary of Dataset <br />

The dataset consists of 9 variables (excluding id's such as name and hashed emails) which are: <br />

`start_time`: Start time and date formatted as `DD/MM/YYYY hh:mm`<br />
`end_time`: End time and date formatted as `DD/MM/YYYY hh:mm`<br />
`original_start_time`: Start time and date formatted in UNIX (ms)<br />
`original_end_time`: End time and date formatted in UNIX (ms)<br />
`experience`: Experience level of player as a string that can be a value of :`Pro`, `Veteran`, `Regular`, `Amateur`, `Beginner`.<br />
`subscribe`: Is the person subscribed to the newsletter? String value of either `True` or `False`<br />
`played_hours`: How many hours the person has played, formatted as numeric float.<br />
`gender`: Gender of respondent, String value. <br />
`Age`: Age of respondent, numeric float <br />



### Initial Data Wrangling and Summary Statistics


#### Libraries:

In [ ]:
### Run this cell before continuing.
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
options(repr.matrix.max.rows = 6)
source("cleanup.R")